In [ ]:
import onnx
from onnx import helper, TensorProto
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
import numpy as np
from qonnx.util.basic import gen_finn_dt_tensor
import onnx.version_converter as vc
from onnx.backend.test.case.node import expect

from qonnx.transformation.infer_shapes import InferShapes


In [302]:
class slice_template():
    def __init__(
            self, input_shape = [4], 
            output_shape = [2], param_shape = [1],
            starts_value = [0], ends_value = [2],
            axes_value = [0], splits_value = [],
            input_tensor = "slice_in", output_tensor = "slice_out",
            dtype = TensorProto.INT64, paramdt = "INT64"):
        
        self.inp_shp = [input_shape]
        self.out_shp = [output_shape]
        self.slice_shape = [param_shape]
        
#         self.idt = DataType[idtype]
#         self.param_dt = DataType[paramdt]
        
        self.input = helper.make_tensor_value_info(input_tensor, dtype, input_shape)
        self.output = helper.make_tensor_value_info(output_tensor, dtype, output_shape)

        self.slice_attr = {}
        self.slice_attr["starts"] = np.array(starts_value,dtype=np.int64)
        self.slice_attr["ends"] = np.array(ends_value,dtype=np.int64)
        self.slice_attr["axes"] = np.array(axes_value,dtype=np.int64)
        if not splits_value == []:
            self.slice_attr["splits"] = np.array(splits_value,dtype=np.int64)
        
        self.opset_version = helper.make_operatorsetid("", 9)

    def make_node(self):
        self.slice_node = helper.make_node(
            "Slice",
            inputs=[self.input.name],
            outputs=[self.output.name],
            **self.slice_attr
        )
        return self.slice_node

    def make_model(self,model_name = "Slice_model.onnx"):

        model_config = {}
        model_config["opset_imports"] = [self.opset_version]

        self.model = ModelWrapper(helper.make_model(
            helper.make_graph(
                [self.slice_node],
                inputs=[self.input],
                outputs=[self.output],
                name="slice_graph"
            ), **model_config)
        )
        self.model.transform(InferShapes())
        self.model.save("onnx_model/%s" %model_name)


In [132]:
x = slice_template(input_shape=[1],output_shape=[1],
                   starts_value=[4], ends_value=[9223372036854775807], axes_value=[4], 
                   splits_value=[1], input_tensor="673", output_tensor="703",
                  dtype=TensorProto.FLOAT)
node = x.make_node()
x.make_model()

In [315]:
tmp_model = ModelWrapper("tinyyolo_infershapes.onnx")

slice_nodes = tmp_model.get_nodes_by_op_type("Slice")
s = slice_nodes[0]
for s in slice_nodes:
    node_inx = tmp_model.get_node_index(s)

    tmp_model.get_node_index(slice_nodes[0])

    vinfo = [tmp_model.get_tensor_valueinfo(inp) for inp in s.input]
    init = [tmp_model.get_initializer(inp) for inp in s.input]
    shapes = [tmp_model.get_tensor_shape(i) for i in s.input]
    t_names = [i for i in s.input]
    key_list = ["input","starts","ends","axes","splits","output"]
    attr = {}
    for k in key_list:
        attr[k] = [None,None,None]

    for ind in range(len(shapes)):
        attr[key_list[ind]] = [t_names[ind],shapes[ind],init[ind]]
    for i in s.output: 
        attr["output"] = [i, tmp_model.get_tensor_shape(i)]
    print("*"*110)
    print(attr)
    print(tmp_model.get_tensor_datatype())
    x = slice_template(input_shape=attr["input"][1],output_shape=attr["output"][1],
                   starts_value=attr["starts"][2], ends_value=attr["ends"][2], axes_value=attr["axes"][2],
                   input_tensor=attr["input"][0], output_tensor=attr["output"][0],
                  dtype=TensorProto.INT64)
    x.make_node()
    x.make_model(s.name+".onnx")

**************************************************************************************************************
{'input': ['735', [4], None], 'starts': ['737', [1], array([0])], 'ends': ['738', [1], array([2])], 'axes': ['736', [1], array([0])], 'splits': [None, None, None], 'output': ['739', [2]]}
**************************************************************************************************************
{'input': ['673', [], None], 'starts': ['675', [1], array([0])], 'ends': ['676', [1], array([2])], 'axes': ['674', [1], array([4])], 'splits': ['677', [1], array([1])], 'output': ['678', []]}
**************************************************************************************************************
{'input': ['673', [], None], 'starts': ['686', [1], array([2])], 'ends': ['687', [1], array([4])], 'axes': ['685', [1], array([4])], 'splits': ['688', [1], array([1])], 'output': ['689', []]}
***********************************************************************************************

In [279]:
attr["ends"][1]

array([2])

In [306]:
x = slice_template(input_shape=attr["input"][1],output_shape=attr["output"][1],
                   starts_value=attr["starts"][2], ends_value=attr["ends"][2], axes_value=attr["axes"][2],
                   input_tensor=attr["input"][0], output_tensor=attr["output"][0],
                  dtype=TensorProto.INT64)
x.make_node()
x.make_model(s.name+".onnx")